<a href="https://colab.research.google.com/github/ayasherby/content/blob/main/Grad_model_Tooth_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!git clone https://github.com/Arnold0210/TEETH-RECOGNITION-WITH-MACHINE-LEARNING


Cloning into 'TEETH-RECOGNITION-WITH-MACHINE-LEARNING'...
remote: Enumerating objects: 32505, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 32505 (delta 19), reused 17 (delta 9), pack-reused 32477
Receiving objects: 100% (32505/32505), 6.29 GiB | 35.78 MiB/s, done.
Resolving deltas: 100% (5099/5099), done.
Updating files: 100% (14166/14166), done.


In [43]:
!pip install opencv-python-headless numpy tqdm

In [45]:
!pip install numpy pandas matplotlib scikit-learn

In [277]:
import os

# Change the working directory to your project root
project_root = '/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment'
os.chdir(project_root)

# Verify the change
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment

In [285]:
!python main.py

PreProcessing Directory Already Exists.
FeatureExtraction Directory Already Exists.
Sampling Directory Already Exists.
Classification Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Already Exists.
HSV Directory Already Exists.
LAB Directory Already Exists.
Lab_ Directory Already Exists.
ResizeImages Directory Already Exists.
Segmentation Directory Already Exists.
BarPlot Directory Already Exists.
BarPlotCh1 Directory Already Exists.
BarPlotCh2 Directory Already Exists.
BarPlotCh3 Directory Already Exists.
Mask Directory Already Exists.
MaskOverlay Directory Already Exists.
Inverse Directory Already Exists.
ResizeImages Directory Already Exists.
RGB2YCbCr Directory Al

# **`Saved codes to run`**

In [ ]:
#Main module

import errno
import os
import sys

import cv2 as cv
import numpy as np
from tqdm import tqdm

import Source.Classification as Cl
import Source.FeatureExtraction as fE
import Source.PreProcessingData as pD
import Source.ReadImages as rI


def show(image):
    cv.imshow('Imagen ', image)
    cv.waitKey(0)
    cv.destroyAllWindows()


class MainClass:
    PROJECT_PATH = os.path.join(os.getcwd(), os.path.pardir)
    PATH_IMAGES_ORIGINAL = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET - Original"))
    PATH_Labels = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"))
    PATH_LabelsXML = os.path.abspath(os.path.join(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"), "LabelsXML"))
    PATH_IMAGES = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET"))
    readimages = None
    preprocessing = None
    featureExtraction = None
    clasification = None

    def __init__(self):
        try:
            if not os.path.exists(self.PROJECT_PATH + 'PreProcessing'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'PreProcessing'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('PreProcessing Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'FeatureExtraction'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('FeatureExtraction Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Sampling'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Sampling'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Sampling Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Classification'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Classification'))
                print('Directory Classification Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Classification Directory Already Exists.')
            else:
                raise
        self.readimages = rI.LoadData(self.PATH_IMAGES_ORIGINAL)
        self.preProcessing = pD.PreProcessingData(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.featureExtraction = fE.FeatureExtraction(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.clasification = Cl.Classification(self.PROJECT_PATH)

    def main_run(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction

        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset ORIGINAL
        # img, name = read.read_One_Image(self.PATH_IMAGES)
        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset RECORTADO
        img, name = read.read_One_Image(self.PATH_IMAGES)

        # Se obtiene las dimensiones de la imagen original
        height_ori, width_ori, depth_ori = img.shape
        # print("Image original shape: \n Height:", height_ori, ", Width:", width_ori)

        # Se realiza un ajuste de tamaño para reducir la imagen a unas dimensiones de 600x400
        img_resize = pp.resize_Image(img, name)

        # La imagen reajustada se convierte de BGR a RGB
        img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
        # Se convierte la imagen de RGB a HSV
        hsv_image = pp.rgb_2_HSV(img_resize, name)
        # Se saca la imagen en pila de tono de rojos
        stack, name = pp.stackColors(hsv_image, name)
        # Se saca los histogramas por imagen para determinar el rango de color de los dientes
        pp.hsv_hist(hsv_image, name)
        #
        # plt.imshow(img_resize)
        # Blur image slightly
        name, blurimage = pp.blurImage(img_resize, name)
        pp.show_mask(blurimage, name)
        pp.overlay_mask(blurimage, img_resize, name)

        # Se obtiene la rueda cromatica de la imágen
        # pp.getChromatiColor(img_resize,name,fe)

        # img_rgb2ycbcr = pp.rgb_2_YCrCb(img_resize, name)
        img_rgb2hsv = pp.rgb_2_HSV(img_resize, name)

    '''easygui.msgbox("Image original shape: \n Height:" + str(height_ori) + "px, Width:" + str(width_ori) + "px" +
                   "\n Image Resize shape: \n Height:" + str(height_res) + "px, Width:" + str(width_res) + "px",
                   image=os.path.join(os.path.join(os.getcwd(), os.path.pardir),
                                      'PreProcessing/ResizeImages/' + name),
                   title="Image Shape - PreProcessing ")'''

    def savebin(self):
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        images, names = read.read_Images(self.PATH_IMAGES_P)
        for image_point, name_point in zip(images, names):
            img_resize = pp.resize_Image(image_point, name_point)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_RGB2GRAY)
            pp.bin(img_resize, name_point)

    def main_alldataset(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        cc = self.clasification
        doption = input(
            'Que desea hacer? \n1. Leer imagenes y obtener caracteristicas'
            '\n2. Leer archivo de caracteristicas y entrenar algoritmo\n')
        option = int(doption)
        print(option)

        if option == 1:
            images, names = read.read_Images(self.PATH_IMAGES)
            bar = tqdm(images, ncols=len(images), unit=' image')
            for image_point, name_point in zip(bar, names):
                bar.set_description("Procesando imagen %s" % name_point)
                # Se reajusta la imagen a un tamaño de 600x400px
                img_resize = pp.resize_Image(image_point, name_point)

                # La imagen reajustada se convierte de BGR a RGB
                img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)

                # Se convierte la imagen de RGB a HSV
                hsv_image = pp.rgb_2_HSV(img_resize, name_point)

                # Se saca la imagen en pila de tono de rojos
                stack, name_point = pp.stackColors(hsv_image, name_point)

                # Se saca los histogramas por imagen para determinar el rango de color de los dientes
                pp.hsv_hist(hsv_image, name_point)

                # Blur image slightly
                name_point, blurimage = pp.blurImage(img_resize, name_point)
                '''file_ = open(os.path.join(self.PROJECT_PATH, 'Pruebas') + name_point + '.txt', "w")
                for i in blurimage:
                    file_.write(str(i))
                file_.close()'''

                # A partir del rango de color, se saca una máscara donde se ubican los dientes y se procede a buscar el contorno más grande dentro del área objetivo.
                mask = pp.findBiggestContour(blurimage, name_point)
                # Se separan los canales de la
                # imágen en RGB
                channelR, channelG, channelB = cv.split(img_resize)
                # Se obtienen los momentos de color de cada espacio de color
                red = fe.getFeaturesVector(channelR, mask)
                green = fe.getFeaturesVector(channelG, mask)
                blue = fe.getFeaturesVector(channelB, mask)
                # colores = ["RED", "GREEN", "BLUE"]

                # Se obtiene una mascara de la imágen compuesta por los valores de los canales donde se encontraron los datos de interes.
                imagen = [red, green, blue]
                # Se coloca la ruta del archivo de caracteristicas
                filefeaturespath = os.path.join(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'), 'features.csv')
                # Se escribe en un archivo las caracteristicas del dataset
                fe.getFeatures(imagen, filefeaturespath, name_point)
                fileLabels = open(os.path.join(self.PATH_Labels, 'Labels.csv'))
                pp.show_mask(blurimage, name_point)
                pp.overlay_mask(blurimage, img_resize, name_point)
        elif option == 2:
            times_execution = -1
            while times_execution <= 0:
                times_execution = int(input('\nIndique la cantidad de veces de ejecución:'))
            bar = tqdm(range(times_execution), unit=' times')
            for i in bar:
                # Se lee el archivo de caracteristicas donde se encuentran los momentos de color
                filefeaturespath = os.path.join(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'), 'features.csv')
                names, features = cc.readfeatures(filefeaturespath)
                labels = cc.readLabels(self.PATH_Labels)
                features_images = features.values
                vals_to_replace = {'a1': '0', 'a2': '1', 'a3': '2', 'a35': '3'}
                tags = ['0', '1', '2', '3']
                target_names = ['a1', 'a2', 'a3', 'a35']
                folds = int(input('\nCantidad de folios a seperarar el conjunto de datos:'))
                test_size = int(input('\n Porcentaje de division del conjunto de datos trianing/test:'))
                X, Y = cc.CrossValidation(features, labels, test_size / 100)
                SVM, DT, KNN = cc.classification(self.PATH_IMAGES, X, Y, folds, tags, target_names, vals_to_replace)
                print('--------- Training ---------')
                for S, D, K in zip(SVM, DT, KNN):

                    matrix_confusion_SVM, report_clasification_SVM, report_scores_SVM = S.split()
                    matrix_confusion_DT, report_clasification_DT, report_scores_DT = D.split()
                    matrix_confusion_KNN, report_clasification_KNN, report_scores_KNN = K.split()
                    print('\n')
                    print('-------SVM------')
                    for report in report_clasification_SVM:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN SVM--------')
                    print(np.mean(report_scores_SVM))
                    print('-------DT------')
                    for report in report_clasification_DT:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN DT--------')
                    print(np.mean(report_scores_DT))

                    print('-------KNN------')
                    for report in report_clasification_KNN:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN KNN--------')
                    print(np.mean(report_scores_KNN))
                    print('--------- TEST ---------')


if __name__ == '__main__':
    tesis: MainClass = MainClass()
    # tesis.main_run()
    tesis.main_alldataset()
    # tesis.savebin()
    print('Se ha finalizado la ejecución del experimento')
    sys.exit(0)

In [ ]:
#Feature Extraction module
import errno
import os
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.cluster import KMeans

import Source.PreProcessingData as pD
import Source.ReadImages as rI


class FeatureExtraction:
    readImages = None
    preProcessing = None

    def __init__(self, PATH_PROJECT, PATH_IMAGES):
        self.path_project = os.path.join(PATH_PROJECT, 'FeatureExtraction')
        self.path_dataset = PATH_IMAGES
        try:
            if not os.path.exists(self.path_project + 'GetColors'):
                self.path_getColor = os.path.join(self.path_project, 'GetColors')
                os.mkdir(self.path_getColor)

                print('ResizeImages Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('ResizeImages Directory Already Exists.')
            else:
                raise
        self.preProcessing = pD.PreProcessingData(PATH_PROJECT, PATH_IMAGES)
        self.readImages = rI.LoadData(PATH_PROJECT)

    def RGB2HEX(self, color):
        return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

    def get_colors(self, src, number_of_colors, show_chart, name):
        modified_image = src.reshape(src.shape[0] * src.shape[1], 3)
        clf = KMeans(n_clusters=number_of_colors)
        labels = clf.fit_predict(modified_image)
        counts = Counter(labels)
        center_colors = clf.cluster_centers_
        # We get ordered colors by iterating through the keys
        ordered_colors = [center_colors[i] for i in counts.keys()]
        hex_colors = [self.RGB2HEX(ordered_colors[i]) for i in counts.keys()]
        rgb_colors = [ordered_colors[i] for i in counts.keys()]

        if show_chart:
            path_image_name = os.path.join(self.path_getColor, name)
            plt.figure(figsize=(8, 6))
            plt.pie(counts.values(), labels=hex_colors, colors=hex_colors)
            plt.savefig(path_image_name)
            plt.show()

        return rgb_colors, hex_colors

    def getFeaturesVector(self, image, mask):
        features = []
        imagecpy = image.copy()
        for i in range(len(mask)):
            for j in range(len(mask[i])):
                if (j == 255):
                    # print(image[i][j])
                    features.append(imagecpy[i][j])
        return features

    def meanVector(self, vector_caracteristicas):
        return np.mean(vector_caracteristicas)

    def varVector(self, vector_caracteristicas):
        return stats.moment(vector_caracteristicas)

    def skewVector(self, vector_caracteristicas):
        return stats.skew(vector_caracteristicas)

    def getFeatures(self, imagen, filefeaturespath, name_point):
        if not (os.path.exists(filefeaturespath) or os.path.isfile(filefeaturespath)):
            filefeatures = open(filefeaturespath, "w")
        else:
            filefeatures = open(filefeaturespath, "a")
        features = []
        for j in range(0, len(imagen)):
            features.append(self.meanVector(imagen[j]))
            features.append(self.varVector(imagen[j]))
            features.append(self.skewVector(imagen[j]))
        filefeatures.write(name_point)
        for item in range(len(features)):
            filefeatures.write(",%.6f" % features[item])
        filefeatures.write("\n")#"," + label +'''
        filefeatures.close()


In [282]:
#classification module
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        print("Length of features",len(features),"\n")
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        #labels_path = os.path.join(labels_path, 'Labels.csv')
        labels_path="/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/Labels/Labels.csv"
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        print("Length of Labels",len(labels),"\n")
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        path_dataset="/content/TEETH-RECOGNITION-WITH-MACHINE-LEARNING/Experiment/DATASET"
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)
            for train_index, test_index in k_folds.split(images_name):
                print("\nLength of features----->",len(features),"\n")
                print("\nLength of Labels----->",len(labels),"\n")
                train_label = []
                test_label = []
                train_features = []
                print("\n-------------------------*************************************\n")
                print("\n Length of train  index",len(train_index), type(train_index),"\n")
                print("\nLength of train_features",len(train_features), type(train_features),"\n")
                print("\n-------------------------*************************************\n")

                test_features = []
                print("\n-------------------------*************************************\n")
                print("\n Length of  test_features",len(test_features) , type(test_features),"\n")
                print("Length of test index",len(test_index) , type(test_index),"\n")
                print("\n-------------------------*************************************\n")

                for i in train_index:
                    print("\n\n <-----Length of i --->",i,type(i),"\n")
                    train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                    print("\n\n <-----****************Heyyyyyyyyyyyyyyyy******** --->",len(train_features),"\n")

                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                    """#Assuming features is a list where each element represents a feature for an image
                    train_features.append(features[i])  # Access feature using index from train_index
                    train_label.append(labels_color[i])  # Access label using index from train_index"""
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:
                      test_features.append(features[i])  # Access feature using index from test_index
                      test_label.append(labels_color[i])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()

## **WorstCase**


In [161]:
#not good
import errno
import os
from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


class Classification:
    def __init__(self, PATH_PROJECT):
        self.path_project = os.path.join(PATH_PROJECT, 'Classification')
        try:
            if not os.path.exists(self.path_project + 'SVM'):
                self.path_getColor = os.path.join(self.path_project, 'SVM')
                os.mkdir(self.path_getColor)
                print('SVM Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('SVM Directory Already Exists.')
            else:
                raise

    def readfeatures(self, features_Path):
        # featuresFile = open(features_Path, "r")
        # featuresData = csv.reader(featuresFile)
        featuresFile = pd.read_csv(features_Path, sep=',', header=None)
        names = featuresFile.iloc[:, 0]
        features = featuresFile.iloc[:, 1:]
        shapefile = featuresFile.shape
        col = []
        for x in range(0, shapefile[1]):
            if x == 0:
                col.append("NAME")
            else:
                col.append("VALOR-" + str(x))
        featuresFile.columns = col
        # print(featuresFile)
        return names, features

    def readLabels(self, labels_path):
        labels_path = os.path.join(labels_path, 'Labels.csv')
        labels = pd.read_csv(labels_path, sep=',', header=[0])
        return labels

    def classificatorSVM(self, features, labels):
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        clf = svm.SVC(gamma='scale')
        clf.fit(X, tags)
        return clf

    def DecisionTree(self, features, labels):
        dt = DecisionTreeClassifier(random_state=30, max_depth=300)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        dt.fit(X, tags)

        return dt

    def KNN(self, features, labels):
        knn = KNeighborsClassifier(n_neighbors=200, algorithm='auto', weights='distance', n_jobs=-1)
        X = []
        for f in features:
            X.append(f)
        tags = []
        for tag in labels:
            tags.append(tag)
        knn.fit(X, tags)
        return knn

    def classification(self, path_dataset, features, labels, n_splits, tags, target_names, vals_to_replace):
        pd.options.mode.chained_assignment = None
        onlyfiles = [f for f in listdir(path_dataset) if
                     isfile(join(path_dataset, f))]
        #print(onlyfiles)
        k_folds = KFold(n_splits=n_splits)
        SVM = []
        KNN = []
        DT = []
        for stage, feature in zip(labels, features):
            stage['Color'] = stage['Color'].map(vals_to_replace)
            labels_color = stage['Color'].to_numpy().tolist()
            images_name = stage['Nombre de la imagen'].to_numpy().tolist()
            svm_training_Score = []
            confusion_matrix_svm = []
            confusion_matrix_dt = []
            confusion_matrix_knn = []
            classification_report_svm = []
            classification_report_dt = []
            classification_report_knn = []
            score_accuracy_SVM = []
            score_accuracy_DT = []
            score_accuracy_KNN = []
            index_images_name = []
            k_folds.get_n_splits(index_images_name)

            for train_index, test_index in k_folds.split(images_name):
                train_label = []
                test_label = []
                train_features = []
                test_features = []
                for i in train_index:
                  try:
                      train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                      train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                  except ValueError:
                      print(f"Warning: '{onlyfiles[i].split('.')[0]}' not found in images_name. Skipping...")
                continue
                    #train_features.append(feature.to_numpy().tolist()[images_name.index(onlyfiles[i].split('.')[0])])
                    # train_features.append(feature.to_numpy()[images_name.str(onlyfiles[i].split('.')[0])])
                    #train_label.append(labels_color[images_name.index(onlyfiles[i].split('.')[0])])
                SVM_Classifier = self.classificatorSVM(train_features, train_label)
                DT_Classifier = self.DecisionTree(train_features, train_label)
                KNN_Classifier = self.KNN(train_features, train_label)

                for i in test_index:
                    test_features.append(feature.to_numpy()[images_name.index(str(onlyfiles[i].split('.')[0]))])
                    test_label.append(labels_color[images_name.index(str(onlyfiles[i].split('.')[0]))])
                predict_label_SVM = SVM_Classifier.predict(test_features)
                predict_label_DT = DT_Classifier.predict(test_features)
                predict_label_KNN = KNN_Classifier.predict(test_features)

                confusionMatrixSVM = confusion_matrix(test_label, predict_label_SVM)
                confusionMatrixDT = confusion_matrix(test_label, predict_label_DT)
                confusionMatrixKNN = confusion_matrix(test_label, predict_label_KNN)

                classification_report_svm.append(
                    classification_report(test_label, predict_label_SVM, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_dt.append(
                    classification_report(test_label, predict_label_DT, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))
                classification_report_knn.append(
                    classification_report(test_label, predict_label_KNN, labels=tags,
                                          target_names=target_names, sample_weight=None, digits=5,
                                          output_dict=False))

                confusion_matrix_svm.append(confusionMatrixSVM)
                confusion_matrix_dt.append(confusionMatrixDT)
                confusion_matrix_knn.append(confusionMatrixKNN)

                score_accuracy_SVM.append(accuracy_score(test_label, predict_label_SVM))
                score_accuracy_DT.append(accuracy_score(test_label, predict_label_DT))
                score_accuracy_KNN.append(accuracy_score(test_label, predict_label_KNN))
            SVM_RESULTS = [confusion_matrix_svm, classification_report_svm, score_accuracy_SVM]
            DT_RESULTS = [confusion_matrix_dt, classification_report_dt, score_accuracy_DT]
            KNN_RESULTS = [confusion_matrix_knn, classification_report_knn, score_accuracy_KNN]
            SVM.append(SVM_RESULTS)
            KNN.append(KNN_RESULTS)
            DT.append(DT_RESULTS)
        return SVM, KNN, DT

    def CrossValidation(self, image, labels, test_size):
        X = []
        Y = []
        X_train, X_test, y_train, y_test = train_test_split(image, labels, test_size=test_size)
        X.append(X_train)
        X.append(X_test)
        Y.append(y_train)
        Y.append(y_test)
        print()
        return X, Y

    def ROC_CURVE(self, label_test, label_score):
        n_classes = label_test.shape
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(label_test[:, i], label_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(label_test.ravel(), label_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr["micro"], tpr["micro"],
                 label='micro-average ROC curve (area = {0:0.2f})'
                       ''.format(roc_auc["micro"]))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                           ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Some extension of Receiver operating characteristic to multi-class')
        plt.legend(loc="lower right")
        plt.show()

In [ ]:
#not good

import errno
import os
import sys

import cv2 as cv
import numpy as np
from tqdm import tqdm

import Source.Classification as Cl
import Source.FeatureExtraction as fE
import Source.PreProcessingData as pD
import Source.ReadImages as rI


def show(image):
    cv.imshow('Imagen ', image)
    cv.waitKey(0)
    cv.destroyAllWindows()


class MainClass:
    PROJECT_PATH = os.path.join(os.getcwd(), os.path.pardir)
    PATH_IMAGES_ORIGINAL = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET - Original"))
    PATH_Labels = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"))
    PATH_LabelsXML = os.path.abspath(os.path.join(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "Labels"), "LabelsXML"))
    PATH_IMAGES = os.path.abspath(
        os.path.join(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir), "DATASET"))
    readimages = None
    preprocessing = None
    featureExtraction = None
    clasification = None

    def __init__(self):
        try:
            if not os.path.exists(self.PROJECT_PATH + 'PreProcessing'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'PreProcessing'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('PreProcessing Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'FeatureExtraction'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('FeatureExtraction Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Sampling'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Sampling'))
                print('Directory Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Sampling Directory Already Exists.')
            else:
                raise
        try:
            if not os.path.exists(self.PROJECT_PATH + 'Classification'):
                os.mkdir(os.path.join(self.PROJECT_PATH, 'Classification'))
                print('Directory Classification Created')
        except OSError as e:
            if e.errno == errno.EEXIST:
                print('Classification Directory Already Exists.')
            else:
                raise
        self.readimages = rI.LoadData(self.PATH_IMAGES_ORIGINAL)
        self.preProcessing = pD.PreProcessingData(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.featureExtraction = fE.FeatureExtraction(self.PROJECT_PATH, self.PATH_IMAGES_ORIGINAL)
        self.clasification = Cl.Classification(self.PROJECT_PATH)

    def main_run(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction

        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset ORIGINAL
        # img, name = read.read_One_Image(self.PATH_IMAGES)
        # Se lee el nombre y la imagen que se encuentre en el PATH del dataset RECORTADO
        img, name = read.read_One_Image(self.PATH_IMAGES)

        # Se obtiene las dimensiones de la imagen original
        height_ori, width_ori, depth_ori = img.shape
        # print("Image original shape: \n Height:", height_ori, ", Width:", width_ori)

        # Se realiza un ajuste de tamaño para reducir la imagen a unas dimensiones de 600x400
        img_resize = pp.resize_Image(img, name)

        # La imagen reajustada se convierte de BGR a RGB
        img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
        # Se convierte la imagen de RGB a HSV
        hsv_image = pp.rgb_2_HSV(img_resize, name)
        # Se saca la imagen en pila de tono de rojos
        stack, name = pp.stackColors(hsv_image, name)
        # Se saca los histogramas por imagen para determinar el rango de color de los dientes
        pp.hsv_hist(hsv_image, name)
        #
        # plt.imshow(img_resize)
        # Blur image slightly
        name, blurimage = pp.blurImage(img_resize, name)
        pp.show_mask(blurimage, name)
        pp.overlay_mask(blurimage, img_resize, name)

        # Se obtiene la rueda cromatica de la imágen
        # pp.getChromatiColor(img_resize,name,fe)

        # img_rgb2ycbcr = pp.rgb_2_YCrCb(img_resize, name)
        img_rgb2hsv = pp.rgb_2_HSV(img_resize, name)

    '''easygui.msgbox("Image original shape: \n Height:" + str(height_ori) + "px, Width:" + str(width_ori) + "px" +
                   "\n Image Resize shape: \n Height:" + str(height_res) + "px, Width:" + str(width_res) + "px",
                   image=os.path.join(os.path.join(os.getcwd(), os.path.pardir),
                                      'PreProcessing/ResizeImages/' + name),
                   title="Image Shape - PreProcessing ")'''

    def savebin(self):
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        images, names = read.read_Images(self.PATH_IMAGES_P)
        for image_point, name_point in zip(images, names):
            img_resize = pp.resize_Image(image_point, name_point)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)
            img_resize = cv.cvtColor(img_resize, cv.COLOR_RGB2GRAY)
            pp.bin(img_resize, name_point)

    def main_alldataset(self):
        # Se declaran las clases para poder utilizar los elementos
        read = self.readimages
        pp = self.preProcessing
        fe = self.featureExtraction
        cc = self.clasification
        doption = input(
            'Que desea hacer? \n1. Leer imagenes y obtener caracteristicas'
            '\n2. Leer archivo de caracteristicas y entrenar algoritmo\n')
        option = int(doption)
        print(option)

        if option == 1:
            images, names = read.read_Images(self.PATH_IMAGES)
            bar = tqdm(images, ncols=len(images), unit=' image')
            for image_point, name_point in zip(bar, names):
                bar.set_description("Procesando imagen %s" % name_point)
                # Se reajusta la imagen a un tamaño de 600x400px
                img_resize = pp.resize_Image(image_point, name_point)

                # La imagen reajustada se convierte de BGR a RGB
                img_resize = cv.cvtColor(img_resize, cv.COLOR_BGR2RGB)

                # Se convierte la imagen de RGB a HSV
                hsv_image = pp.rgb_2_HSV(img_resize, name_point)

                # Se saca la imagen en pila de tono de rojos
                stack, name_point = pp.stackColors(hsv_image, name_point)

                # Se saca los histogramas por imagen para determinar el rango de color de los dientes
                pp.hsv_hist(hsv_image, name_point)

                # Blur image slightly
                name_point, blurimage = pp.blurImage(img_resize, name_point)
                '''file_ = open(os.path.join(self.PROJECT_PATH, 'Pruebas') + name_point + '.txt', "w")
                for i in blurimage:
                    file_.write(str(i))
                file_.close()'''

                # A partir del rango de color, se saca una máscara donde se ubican los dientes y se procede a buscar el contorno más grande dentro del área objetivo.
                mask = pp.findBiggestContour(blurimage, name_point)
                # Se separan los canales de la
                # imágen en RGB
                channelR, channelG, channelB = cv.split(img_resize)
                # Se obtienen los momentos de color de cada espacio de color
                red = fe.getFeaturesVector(channelR, mask)
                green = fe.getFeaturesVector(channelG, mask)
                blue = fe.getFeaturesVector(channelB, mask)
                # colores = ["RED", "GREEN", "BLUE"]

                # Se obtiene una mascara de la imágen compuesta por los valores de los canales donde se encontraron los datos de interes.
                imagen = [red, green, blue]
                # Se coloca la ruta del archivo de caracteristicas
                filefeaturespath = os.path.join(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'), 'features.csv')
                # Se escribe en un archivo las caracteristicas del dataset
                fe.getFeatures(imagen, filefeaturespath, name_point)
                fileLabels = open(os.path.join(self.PATH_Labels, 'Labels.csv'))
                pp.show_mask(blurimage, name_point)
                pp.overlay_mask(blurimage, img_resize, name_point)
        elif option == 2:
            times_execution = -1
            while times_execution <= 0:
                times_execution = int(input('\nIndique la cantidad de veces de ejecución:'))
            bar = tqdm(range(times_execution), unit=' times')
            for i in bar:
                # Se lee el archivo de caracteristicas donde se encuentran los momentos de color
                filefeaturespath = os.path.join(os.path.join(self.PROJECT_PATH, 'FeatureExtraction'), 'features.csv')
                names, features = cc.readfeatures(filefeaturespath)
                labels = cc.readLabels(self.PATH_Labels)
                features_images = features.values
                vals_to_replace = {'a1': '0', 'a2': '1', 'a3': '2', 'a35': '3'}
                tags = ['0', '1', '2', '3']
                target_names = ['a1', 'a2', 'a3', 'a35']
                folds = int(input('\nCantidad de folios a seperarar el conjunto de datos:'))
                test_size = int(input('\n Porcentaje de division del conjunto de datos trianing/test:'))
                X, Y = cc.CrossValidation(features, labels, test_size / 100)
                SVM, DT, KNN = cc.classification(self.PATH_IMAGES, X, Y, folds, tags, target_names, vals_to_replace)
                print('--------- Training ---------')
                for S, D, K in zip(SVM, DT, KNN):

                    matrix_confusion_SVM, report_clasification_SVM, report_scores_SVM = S.split()
                    matrix_confusion_DT, report_clasification_DT, report_scores_DT = D.split()
                    matrix_confusion_KNN, report_clasification_KNN, report_scores_KNN = K.split()
                    print('\n')
                    print('-------SVM------')
                    for report in report_clasification_SVM:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN SVM--------')
                    print(np.mean(report_scores_SVM))
                    print('-------DT------')
                    for report in report_clasification_DT:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN DT--------')
                    print(np.mean(report_scores_DT))

                    print('-------KNN------')
                    for report in report_clasification_KNN:
                        print(report)
                        for item in report:
                            print(report[item])
                    print('--------MEAN KNN--------')
                    print(np.mean(report_scores_KNN))
                    print('--------- TEST ---------')


if __name__ == '__main__':
    tesis: MainClass = MainClass()
    # tesis.main_run()
    tesis.main_alldataset()
    # tesis.savebin()
    print('Se ha finalizado la ejecución del experimento')
    sys.exit(0)
